In [ ]:
# if hit plotting library issues, try resetting LD path for julia
# can set in ~/.local/share/jupyter/kernels/
@assert ENV["LD_LIBRARY_PATH"] == ""
import Pkg
Pkg.activate(;temp=true)
Pkg.add("Revise")
import Revise

# Pkg.develop(path="/global/cfs/projectdirs/m4269/cjprybol/Mycelia")
# Pkg.develop(path="../../..")
Pkg.develop(path="$(homedir())/workspace/Mycelia")
import Mycelia

pkgs = String[
    "DataFrames"
]
Pkg.add(pkgs)
for pkg in pkgs
    eval(Meta.parse("import $pkg"))
end

In [ ]:
db = "nt"

In [ ]:
path_to_db = joinpath(homedir(), "workspace", "blastdb", db)
# path_to_db = Mycelia.download_blast_db(db=db, source="ncbi")

In [ ]:
# compressed_fasta_export = Mycelia.export_blast_db(path_to_db = path_to_db)
compressed_fasta_export = path_to_db * ".fna.gz"

In [ ]:
cmd, outfile = Mycelia.minimap_index(fasta=compressed_fasta_export, mem_gb=Mycelia.NERSC_MEM, mapping_type="map-hifi", threads=Mycelia.NERSC_CPU, as_string=true)

In [ ]:
# if !isfile(outfile)
#     println(outfile)
#     Mycelia.nersc_sbatch_premium(
#         job_name=basename(outfile),
#         mail_user="cameron.prybol@gmail.com",
#         logdir=mkpath("$(homedir())/workspace/slurmlogs"),
#         mem_gb=Mycelia.NERSC_MEM,
#         cpus_per_task=Mycelia.NERSC_CPU,
#         cmd=cmd)
# end

In [ ]:
# try mapping something to it